In [1]:
! df

Filesystem                 1K-blocks     Used  Available Use% Mounted on
/dev/mapper/rootvg-lvroot    5095040    50576    4797400   2% /
devtmpfs                    65905892        0   65905892   0% /dev
tmpfs                       65931420        0   65931420   0% /dev/shm
tmpfs                       65931420  4186952   61744468   7% /run
tmpfs                       65931420        0   65931420   0% /sys/fs/cgroup
/dev/mapper/rootvg-lvusr     3997376  3011812     759468  80% /usr
/dev/vda1                     475736   137495     309160  31% /boot
/dev/mapper/rootvg-lvhome    3997376    22160    3749120   1% /home
/dev/mapper/rootvg-lvtmp    15350728   192592   14470680   2% /tmp
/dev/mapper/rootvg-lvvar    30832548 19371372   10144588  66% /var
/dev/mapper/rootvg-lvopt  1999631152 78642028 1839690456   5% /opt
tmpfs                       13186284        0   13186284   0% /run/user/0
cm_processes                65931420    51940   65879480   1% /run/cloudera-scm-agent/process
tmpfs    

In [2]:
import os
import sys
 
spark_home = '/opt/cloudera/parcels/SPARK2/lib/spark2'
os.environ['SPARK_HOME'] = spark_home
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'
 
sys.path.insert(0, os.path.join (spark_home,'python'))
 
sys.path.insert(0, os.path.join (spark_home,'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkContext, SparkConf, HiveContext
 
conf = SparkConf().setAppName('SparkContextExample')\
    .setMaster("yarn-client")\
    .set('spark.dynamicAllocation.enabled', 'false')\
    .set("hive.metastore.uris", "thrift://februs6.lab.df.sbrf.ru:9083")\
    .set("hive.metastore.uris", "thrift://februs7.lab.df.sbrf.ru:9083")
# Пример кастомизации параметров Spark. По умолчанию параметры берутся из конфигурации Cloudera Manager.
# .set('spark.local.dir', 'sparktmp')\ # Директория для временных файлов Spark
# .set('spark.executor.memory','5g')\ # Объём памяти для выполнения задач Spark
# .set('spark.driver.maxResultSize','5g')\ # Максимальный объём памяти для драйвера Spark
# .set('spark.executor.instances', '1')\ # Количество экземпляров/инстансов для выполнения задач Spark
# .set('spark.executor.cores', '8')\ # Количество ядер, задействованных для выполнения задач Spark
# .set('spark.port.maxRetries', '150')\ # Максимальное количество попыток привязки к порту до отказа


In [3]:
sc = SparkContext.getOrCreate(conf=conf)

In [4]:
sqlc = HiveContext(sc)

In [5]:
sqlc.sql("""select max(protocol_dttm) from custom_cb_p4d_zakupki.tender_participants""").show()

+-------------------+
| max(protocol_dttm)|
+-------------------+
|2019-12-11 07:59:00|
+-------------------+



In [6]:
sc.stop()

In [7]:
sys.path.insert(0, '/home/patrakova1-es_ca-sbrf-ru/ds/spark/')

In [8]:
from spark_helper import SparkHelper

In [9]:
helper=SparkHelper()
sc,sql_context=helper.create_sql_context(port='4442', name='dataset', instances=20)

Start
Context ready: <SparkContext master=yarn-client appName=dataset_4442>


In [11]:
sql_context.sql("""select max(protocol_dttm), min(protocol_dttm) from custom_cb_p4d_zakupki.tender_participants""").show()

+-------------------+-------------------+
| max(protocol_dttm)| min(protocol_dttm)|
+-------------------+-------------------+
|2019-12-11 07:59:00|2017-01-26 17:09:33|
+-------------------+-------------------+



In [12]:
sc.stop()